<a href="https://colab.research.google.com/github/kinn14/health_fact_check_classification/blob/main/Copy_of_NLP_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version
!sudo apt-get update -y
!sudo apt-get install python3.9

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

                                          #check python version
!python --version
# install pip for new python
!sudo apt-get install python3.9-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

# credit of these last two commands blongs to @Erik
# install colab's dependenciesā
!python -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

# link to the old google package
!ln -s /usr/local/lib/python3.8/dist-packages/google \
       /usr/local/lib/python3.9/dist-packages/google

Python 3.11.11
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,285 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,647 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,625 kB]
Get:13 http://archive.ubuntu.

In [ ]:
!python --version

Python 3.11.11


Restart runtime

# Pytorch Model Implementation

## Install Necessary Libraries

In [ ]:
!pip install transformers datasets torch

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the health_fact dataset
dataset = load_dataset("ImperialCollegeLondon/health_fact", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.61k [00:00<?, ?B/s]

health_fact.py:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1235 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1225 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

# Load a tokenizer
model_name = "bert-base-uncased"  # Replace with your preferred model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["claim"], examples["explanation"], truncation=True, padding="max_length", max_length=512)

encoded_dataset = dataset.map(preprocess_function, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1235 [00:00<?, ? examples/s]

Map:   0%|          | 0/1225 [00:00<?, ? examples/s]

In [ ]:
# Map string labels to integers
label_map = {-1:2, 0:0, 1:1,2:2,3:3}
def encode_labels(examples):
    examples["label"] = [label_map[label] for label in examples["label"]]
    return examples

encoded_dataset = encoded_dataset.map(encode_labels, batched=True)

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1235 [00:00<?, ? examples/s]

Map:   0%|          | 0/1225 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
np.unique(dataset['train']['label'])
np.unique(encoded_dataset['train']['label'])

array([0, 1, 2, 3])

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load a model for classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)  # 3 labels: true, false, mixture

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader

# Create DataLoaders
train_dataloader = DataLoader(encoded_dataset["train"], shuffle=True, batch_size=16)
val_dataloader = DataLoader(encoded_dataset["validation"], batch_size=16)

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    # Add this line for debugging
    # report_to="none",  # Disable reporting to avoid potential conflicts
    fp16=False  # Disable mixed precision to avoid potential issues

)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-b69c513947eb>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.543400,0.626398
2,0.949400,0.712870


TrainOutput(global_step=4916, training_loss=0.6325932601004524, metrics={'train_runtime': 788.9356, 'train_samples_per_second': 24.925, 'train_steps_per_second': 6.231, 'total_flos': 5173908699807744.0, 'train_loss': 0.6325932601004524, 'epoch': 2.0})

In [ ]:
metrics = trainer.evaluate(encoded_dataset["test"])
print(metrics)

{'eval_loss': 0.8570383787155151, 'eval_runtime': 10.0663, 'eval_samples_per_second': 122.686, 'eval_steps_per_second': 30.696, 'epoch': 2.0}


In [ ]:
model.save_pretrained("./health_fact_model")
tokenizer.save_pretrained("./health_fact_model")

('./health_fact_model/tokenizer_config.json',
 './health_fact_model/special_tokens_map.json',
 './health_fact_model/vocab.txt',
 './health_fact_model/added_tokens.json',
 './health_fact_model/tokenizer.json')

In [ ]:
from transformers import pipeline

# Load the fine-tuned model
classifier = pipeline("text-classification", model="./health_fact_model", tokenizer=tokenizer)

# Test it on a new claim
result = classifier("Eating spicy food is not good for body.")
print(result)

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.841720461845398}]
